In [1]:
import numpy as np
import jax
import jax.numpy as jnp
from jaxlie import SE3, SO3
import jax_dataclasses as jdc
from functools import partial
import PyCeres

from sdf_world.sdf_world import *
from sdf_world.robots import *
from sdf_world.util import *

In [2]:
world = SDFWorld()
panda_model = RobotModel(PANDA_URDF, PANDA_PACKAGE)
panda = Robot(world.vis, "panda", panda_model, alpha=0.5)
panda.reduce_dim([7, 8], [0.04, 0.04])

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [3]:
#utility
def to_posevec(pose:SE3):
    return jnp.hstack([pose.translation(), pose.rotation().log()])

def make_pose():
    return SE3.from_rotation_and_translation(
        SO3(np.random.random(4)).normalize(),
        np.random.uniform([-0.3,-0.5,0.3],[0.6, 0.5, 0.8])
    )

# Kinematics
def get_rotvec_angvel_map(v):
    def skew(v):
        v1, v2, v3 = v
        return jnp.array([[0, -v3, v2],
                        [v3, 0., -v1],
                        [-v2, v1, 0.]])
    vmag = jnp.linalg.norm(v)
    vskew = skew(v)
    return jnp.eye(3) \
        - 1/2*skew(v) \
        + vskew@vskew * 1/vmag**2 * (1-vmag/2 * jnp.sin(vmag)/(1-jnp.cos(vmag)))

@jax.jit
def get_ee_fk_jac(q):
    # outputs ee_posevec and analytical jacobian
    fks = panda_model.fk_fn(q)
    p_ee = fks[-1][-3:]
    rotvec_ee = SO3(fks[-1][:4]).log()
    E = get_rotvec_angvel_map(rotvec_ee)
    jac = []
    for posevec in fks[1:8]:
        p_frame = posevec[-3:]
        rot_axis = SE3(posevec).as_matrix()[:3, 2]
        lin_vel = jnp.cross(rot_axis, p_ee - p_frame)
        jac.append(jnp.hstack([lin_vel, rot_axis]))
    jac = jnp.array(jac).T
    jac = jac.at[3:, :].set(E @ jac[3:, :])
    return jnp.hstack([p_ee, rotvec_ee]), jac



In [4]:
frame = Frame(world.vis, "frame")
frame_d = Frame(world.vis, "frame_d")

In [6]:
def make_pose():
    return SE3.from_rotation_and_translation(
        SO3(np.random.random(4)).normalize(),
        np.random.uniform([-0.3,-0.5,0.3],[0.6, 0.5, 0.8])
    )
def to_posevec(pose:SE3):
    return jnp.hstack([pose.translation(), pose.rotation().log()])
def to_SE3(posevec:Array):
    rot = SO3.exp(posevec[3:])
    return SE3.from_rotation_and_translation(rot, posevec[:3])

In [87]:
import time
class PoseError(PyCeres.CostFunction):
    def __init__(self):
        super().__init__()
        self.set_num_residuals(6)
        self.set_parameter_block_sizes([6])
        self.goal = None
        self.weight_mat = None

    def set_goal(self, posevec):
        self.goal = posevec
    
    def set_weight(self, weight):
        self.weight_mat = np.diag(np.sqrt(weight))

    def Evaluate(self, parameters, residuals, jacobians):
        posevec = parameters[0] #vector
        residuals[:] = self.weight_mat @ (self.goal - posevec)
        if (jacobians != None):
            jac = - np.eye(6)
            jacobians[0][:] = (self.weight_mat @ jac).flatten()
        return True

class KinError(PyCeres.CostFunction):
    def __init__(self):
        super().__init__()
        self.set_num_residuals(6)
        self.set_parameter_block_sizes([6, 7])
        self.weight_mat = None
    
    def set_weight(self, weight):
        self.weight_mat = np.diag(np.sqrt(weight))

    def Evaluate(self, parameters, residuals, jacobians):
        posevec, q  = parameters[0], parameters[1] #vector
        ee, jac_q = get_ee_fk_jac(q)
        residuals[:] = self.weight_mat @ (posevec - ee)
        if (jacobians != None):
            jacobians[0][:] = (self.weight_mat @ np.eye(6)).flatten()
            jacobians[1][:] = - (self.weight_mat @ jac_q).flatten()
        
        frame.set_pose(to_SE3(posevec))
        panda.set_joint_angles(q)
        time.sleep(0.1)
        return True

joint_lim_viol = lambda q: jnp.clip(q - panda.ub, a_min=0.).sum()
grad_joint_lim_viol = jax.grad(joint_lim_viol)
class JointViolation(PyCeres.CostFunction):
    def __init__(self):
        super().__init__()
        self.set_num_residuals(1)
        self.set_parameter_block_sizes([7])
    
    def Evaluate(self, parameters, residuals, jacobians):
        q = parameters[0]
        residuals[0] = joint_lim_viol(q)
        if (jacobians != None):
            jacobians[0][:] = grad_joint_lim_viol(q)
        return True

In [60]:
pose_d = make_pose()
frame_d.set_pose(pose_d)
posevec_d = to_posevec(pose_d)

In [69]:
pose_weight = np.array([1, 1, 1, 0.3, 0.3, 0.3])
#q_weight = np.ones(7)

In [88]:
feature_pose = PoseError()
feature_pose.set_goal(posevec_d)
feature_pose.set_weight(pose_weight)
feature_kin = KinError()
feature_kin.set_weight(pose_weight/10)
feature_joint_viol = JointViolation()

In [1]:
p = np.zeros(6)
q = panda.neutral.copy()
frame.set_pose(to_SE3(p))
panda.set_joint_angles(q)

problem = PyCeres.Problem()
problem.AddResidualBlock(feature_pose, None, p)
problem.AddResidualBlock(feature_kin, None, p, q)
#problem.AddResidualBlock(feature_joint_viol, None, q)
options = PyCeres.SolverOptions()
options.linear_solver_type = PyCeres.LinearSolverType.DENSE_QR
options.parameter_tolerance = 1e-4
options.minimizer_progress_to_stdout = True
summary = PyCeres.Summary()

NameError: name 'np' is not defined

In [101]:
PyCeres.Solve(options, problem, summary)

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  1.048614e+00    0.00e+00    7.99e-01   0.00e+00   0.00e+00  1.00e+04        0    1.11e-01    1.11e-01
   1  3.391999e-02    1.01e+00    6.16e-02   4.61e+00   9.68e-01  3.00e+04        1    2.15e-01    3.26e-01
   2  2.451733e-01   -2.11e-01    6.16e-02   9.16e+00  -6.23e+00  1.50e+04        1    1.14e-01    4.40e-01
   3  2.568029e-01   -2.23e-01    6.16e-02   8.76e+00  -6.57e+00  3.75e+03        1    1.05e-01    5.45e-01
   4  1.232645e-01   -8.93e-02    6.16e-02   7.02e+00  -2.64e+00  4.69e+02        1    1.14e-01    6.59e-01
   5  7.157870e-02   -3.77e-02    6.16e-02   3.23e+00  -1.13e+00  2.93e+01        1    1.08e-01    7.67e-01
   6  1.579268e-02    1.81e-02    2.56e-02   1.53e+00   5.69e-01  2.94e+01        1    2.15e-01    9.83e-01
   7  1.285793e-02    2.93e-03    3.28e-02   1.33e+00   1.90e-01  2.37e+01        1    2.36e-01    1.22e+00
   8  1.173596e-03    1.17e-

In [102]:
joint_lim_viol(q)

Array(0.06372829, dtype=float32)

: 

In [95]:
q < panda.ub

array([ True,  True,  True,  True,  True, False,  True])

In [35]:
print(summary.BriefReport())

Ceres Solver Report: Iterations: 2, Initial cost: 7.244982e-01, Final cost: 7.239250e-09, Termination: CONVERGENCE
